In [1]:
from __future__ import division, print_function
import os
os.chdir('/Users/tommyalford/LIGO_research/bilby_relative_binning_github/bilby_relative_binning')

import bilby
import numpy as np
import pickle 

import importlib
importlib.reload(bilby)

<module 'bilby' from '/Users/tommyalford/LIGO_research/bilby_relative_binning_github/bilby_relative_binning/bilby/__init__.py'>

In [1]:
"""
This tutorial includes advanced specifications
for analysing binary neutron star event data.
Here GW170817 is used as an example.
"""
outdir = 'test_outdir'
label = 'GW170817'
time_of_event = bilby.gw.utils.get_event_time(label)
bilby.core.utils.setup_logger(outdir=outdir, label=label)
# GET DATA FROM INTERFEROMETER
# include 'V1' for appropriate O2 events
interferometer_names = ['H1']#, 'L1', 'V1']
duration = 32
roll_off = 0.2  # how smooth is the transition from no signal
# to max signal in a Tukey Window.
psd_offset = -512  # PSD is estimated using data from
# `center_time+psd_offset` to `center_time+psd_offset + psd_duration`
# This determines the time window used to fetch open data.
psd_duration = 1024
coherence_test = False  # coherence between detectors
filter_freq = None  # low pass filter frequency to cut signal content above
# Nyquist frequency. The condition is 2 * filter_freq >= sampling_frequency


# All keyword arguments are passed to
# `gwpy.timeseries.TimeSeries.fetch_open_data()'
kwargs = {}
# Data are stored by LOSC at 4096 Hz, however
# there may be event-related data releases with a 16384 Hz rate.
kwargs['sample_rate'] = 4096
# For O2 events a "tag" is required to download the data.
# CLN = clean data; C02 = raw data
kwargs['tag'] = 'C02'


# This step generally takes more longer to run. If you want you can pickle this
# data to load it more quickly:  

# interferometers = bilby.gw.detector.get_event_data(
#     label,
#     interferometer_names=interferometer_names,
#     duration=duration,
#     roll_off=roll_off,
#     psd_offset=psd_offset,
#     psd_duration=psd_duration,
#     cache=True,
#     filter_freq=filter_freq,
#     **kwargs)

interferometers = pickle.load(open('interferometer_data.pkl', 'rb'))

# CHOOSE PRIOR FILE
prior = bilby.gw.prior.BNSPriorDict(filename='GW170817.prior')
deltaT = 0.1
prior['geocent_time'] = bilby.core.prior.Uniform(
    minimum=time_of_event - deltaT / 2,
    maximum=time_of_event + deltaT / 2,
    name='geocent_time',
    latex_label='$t_c$',
    unit='$s$')


# GENERATE WAVEFORM
# OVERVIEW OF APPROXIMANTS:
# https://www.lsc-group.phys.uwm.edu/ligovirgo/cbcnote/Waveforms/Overview
duration = None  # duration and sampling frequency will be overwritten
# to match the ones in interferometers.
sampling_frequency = kwargs['sample_rate']
start_time = 0  # set the starting time of the time array
waveform_arguments = {'reference_frequency': 20}

source_model = bilby.gw.source.lal_binary_neutron_star
convert_bns = bilby.gw.conversion.convert_to_lal_binary_neutron_star_parameters
waveform_generator = bilby.gw.WaveformGenerator(
    duration=duration,
    sampling_frequency=sampling_frequency,
    start_time=start_time,
    frequency_domain_source_model=source_model,
    parameter_conversion=convert_bns,
    waveform_arguments=waveform_arguments,)

init_params = {
    'chirp_mass': 1.1977, 'symmetric_mass_ratio': .244, 
    'a_1': 0, 'a_2': 0, 'tilt_1': 0, 'tilt_2': 0,
    'phi_12': 0, 'phi_jl': 0, 'luminosity_distance': 50,
    'dec': 0, 'ra': 0, 'theta_jn': 0, 'psi': 0, 'phase': 0,
    #'lambda_1': 0, 'lambda_2': 0, 
    'geocent_time': 11870882.3848135}

parameter_bounds = {
    'chirp_mass': [1.18, 1.2], 'symmetric_mass_ratio': [.2, .2499],
    'a_1': [0, .05], 'a_2': [0, .05], 'tilt_1': [0, 2 * np.pi],
    'tilt_2': [0, 2 * np.pi], 'phi_12': [0, 2 * np.pi],
    'phi_jl': [0, 2 * np.pi], 'luminosity_distance': [10, 100],
    'dec': [0, 2 * np.pi], 'ra': [0, 2 * np.pi],
    'theta_jn': [0, 2 * np.pi], 'psi': [0, np.pi],
    'phase': [0, 2 * np.pi], 
    #'lambda_1': [0, 5000], 'lambda_2': [0, 5000],
    'geocent_time': [1187008881.3848135, 1187008883.3848135]}

# CHOOSE LIKELIHOOD FUNCTION
# Time marginalisation uses FFT.
# Distance marginalisation uses a look up table calculated at run time.
# Phase marginalisation is done analytically using a Bessel function.
likelihood = bilby.gw.likelihood.RelativeBinningGravitationalWaveTransient(
    interferometers,
    waveform_generator,
    initial_parameters=init_params, 
    parameter_bounds=parameter_bounds, 
    debug=True)

# RUN SAMPLER
# Implemented Samplers:
# LIST OF AVAILABLE SAMPLERS: Run -> bilby.sampler.implemented_samplers
# conversion function = bilby.gw.conversion.generate_all_bns_parameters
npoints = 512
sampler = 'dynesty'
result = bilby.run_sampler(
    likelihood,
    prior,
    outdir=outdir,
    label=label,
    sampler=sampler,
    npoints=npoints,
    use_ratio=False,
    conversion_function=bilby.gw.conversion.generate_all_bns_parameters)

# result.plot_corner()

#+begin_example
14:00 bilby INFO    : Waveform generator initiated with
  frequency_domain_source_model: bilby.gw.source.lal_binary_neutron_star
  time_domain_source_model: None
  parameter_conversion: bilby.gw.conversion.convert_to_lal_binary_neutron_star_parameters
14:00 bilby INFO    : Running for label 'GW170817', output will be saved to 'test_outdir'
14:00 bilby INFO    : Performing redundancy check using BBHPriorDict(self).test_redundancy
14:00 bilby INFO    : Performing redundancy check using BBHPriorDict(self).test_redundancy
14:00 bilby INFO    : Performing redundancy check using BBHPriorDict(self).test_redundancy
14:00 bilby INFO    : Performing redundancy check using BBHPriorDict(self).test_redundancy
14:00 bilby INFO    : Performing redundancy check using BBHPriorDict(self).test_redundancy
14:00 bilby INFO    : Performing redundancy check using BBHPriorDict(self).test_redundancy
14:00 bilby INFO    : Performing redundancy check using BBHPriorDict(self).test_redundancy
14:00


# Other Checks



In [1]:
likelihood.bin_inds

array([  640,   664,   688,   715,   745,   778,   814,   854,
         899,   950,  1007,  1073,  1150,  1239,  1344,  1470,
        1621,  1808,  2039,  2330,  2697,  3161,  3736,  4431,
        5236,  6126,  7073,  8047,  9029, 10004, 10962, 11900,
       12815, 13706, 14573, 15418, 16239, 17040, 17820, 18581,
       19323, 20049, 20758, 21451, 22131, 22796, 23448, 24088,
       24715, 25331, 25937, 26532, 27116, 27692, 28259, 28817,
       29367, 29908, 30443, 30969, 31488, 32000])

In [1]:
freq_mas = np.array([True, True, True, True, True])
test = np.array([1, 2, 3, 4, 5])
test[freq_mas]

array([1, 2, 3, 4, 5])

In [1]:
test = freq_mas.copy()
test[[1, 3]] = True

In [1]:
np.array([1, 2, 3, 4, 5])[test]

array([2, 4])

In [1]:
likelihood.bin_freqs

#+begin_example
array([  20.        ,   20.7252145 ,   21.48962979,
         22.33244665,   23.27326547,   24.29248585,
         25.40970819,   26.66413328,   28.07536151,
         29.66299326,   31.46622932,   33.52427049,
         35.91551831,   38.69877398,   41.99163983,
         45.91171823,   50.6550131 ,   56.47632953,
         63.70887418,   72.78385568,   84.26968539,
         98.75437509,  116.74753495,  138.4647693 ,
        163.61207224,  191.4250285 ,  221.00202004,
        251.46102922,  282.13564271,  312.59465189,
        342.54405088,  371.84663693,  400.44360887,
        428.29576592,  455.40310806,  481.7852357 ,
        507.46174923,  532.47184944,  556.85473709,
        580.6300126 ,  603.83687674,  626.51453029,
        648.66297326,  670.34100682,  691.56823136,
        712.36424728,  732.72905458,  752.72145443,
        772.32184644,  791.58903178,  810.50341007,
        829.10418208,  847.37174743,  865.3649073 ,
        883.08366167,  900.50841017,  917.697953

In [1]:
np.invert(interferometers[0].strain_data.frequency_mask).sum()

0

In [1]:
interferometers[0].strain_data.frequency_array

array([0.00000000e+00, 3.12500000e-02, 6.25000000e-02, ...,
       2.04793750e+03, 2.04796875e+03, 2.04800000e+03])

nil:END:

